# <p style="text-align: center;"> Starbucks Capstone Challenge

<img src="./Starbucks_Rewards_App.png" width="400" height="300">

In [12]:
import pandas as pd
import numpy as np
import math
import json
import seaborn as sns
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.estimator import Estimator
import plotly.express as px
import matplotlib.pyplot as plt
from dateutil import parser
from datetime import datetime
%matplotlib inline

## Training a DeepAR Estimator

In [13]:
# session, role, bucket
sagemaker_session = sagemaker.Session()
role = get_execution_role()

bucket = sagemaker_session.default_bucket()
prefix = 'deepar-starbucks-capstone'

In [10]:
image_name = get_image_uri(boto3.Session().region_name, # get the region
                           repo_name='forecasting-deepar')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


#### Instantiate an Estimator

In [15]:
# dir to save model artifacts
s3_output_path = "s3://{}/{}/output".format(bucket, prefix)

# instantiate a DeepAR estimator
estimator = Estimator(sagemaker_session=sagemaker_session,
                      image_uri=image_name,
                      role=role,
                      train_instance_count=1,
                      train_instance_type='ml.c4.xlarge',
                      output_path=s3_output_path
                      )

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Setting Hyperparameters

 - epochs
 - time_freq
 - predictions_length
 - context_length

In [16]:
freq='H'
context_length=28 # same as prediction_length
prediction_length = 28

hyperparameters = {
    "epochs": "50",
    "time_freq": freq,
    "prediction_length": str(prediction_length),
    "context_length": str(context_length),
    "num_cells": "50",
    "num_layers": "2",
    "mini_batch_size": "128",
    "learning_rate": "0.001",
    "early_stopping_patience": "10"
}

In [17]:
# set the hyperparams
estimator.set_hyperparameters(**hyperparameters)

### Training Job - Falta pegar o train_path e o test_path

In [ ]:
%%time
# train and test channels
data_channels = {
    "train": train_path,
    "test": test_path
}

# fit the estimator
estimator.fit(inputs=data_channels)